<a href="https://colab.research.google.com/github/rahiakela/genai-research-and-practice/blob/main/campusx-langchain/03_vector_store_chroma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
!pip install -qU langchain-google-genai

In [ ]:
!pip install langchain chromadb tiktoken pypdf langchain-community

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma

from langchain.schema import Document

import os
from google.colab import userdata

In [ ]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

## Preparing Data

In [ ]:
# Create LangChain documents for IPL players
doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )


In [ ]:
docs = [doc1, doc2, doc3, doc4, doc5]

## Strore Document

In [ ]:
vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory='my_chroma_db',
    collection_name='sample'
)

In [ ]:
# add documents
vector_store.add_documents(docs)

['2aae31a9-71fd-4f64-988b-b57b05235522',
 '879eb886-4644-4176-b93b-bbfc1bdd0bf0',
 '3b927a1a-39ee-43dd-99bd-74e8637a703f',
 '6e08b73b-e36d-4907-aa29-276bb9ebeed2',
 'bc1a0a83-0809-4779-908e-17ee3a77fd26']

In [ ]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['2aae31a9-71fd-4f64-988b-b57b05235522',
  '879eb886-4644-4176-b93b-bbfc1bdd0bf0',
  '3b927a1a-39ee-43dd-99bd-74e8637a703f',
  '6e08b73b-e36d-4907-aa29-276bb9ebeed2',
  'bc1a0a83-0809-4779-908e-17ee3a77fd26'],
 'embeddings': array([[-0.05562957, -0.0203272 , -0.01269856, ...,  0.00051751,
         -0.02111294, -0.01154474],
        [-0.00921848, -0.02626629, -0.03133413, ...,  0.00726502,
         -0.04161012, -0.00883735],
        [-0.03262324,  0.01831091, -0.0045399 , ...,  0.00915938,
         -0.02804055, -0.03489326],
        [-0.0146971 , -0.01844227, -0.02503126, ...,  0.01035003,
         -0.01682558, -0.01603061],
        [-0.03304848, -0.03280932, -0.02324565, ..., -0.00354889,
         -0.00507506, -0.00217535]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the most successful ca

## Search documents

In [ ]:
# search documents
vector_store.similarity_search(
    query='Who among these are a bowler?',
    k=2
)

[Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.')]

In [ ]:
# search with similarity score
vector_store.similarity_search_with_score(
    query='Who among these are a bowler?',
    k=2
)

[(Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.8699021339416504),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.9354298114776611)]

In [ ]:
# meta-data filtering
vector_store.similarity_search_with_score(
    query="",
    filter={"team": "Chennai Super Kings"}
)

[(Document(metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  1.1392717361450195),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  1.1505862474441528)]

In [ ]:
# update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id='09a39dc6-3ba6-4ea7-927e-fdda591da5e4', document=updated_doc1)


In [ ]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['2aae31a9-71fd-4f64-988b-b57b05235522',
  '879eb886-4644-4176-b93b-bbfc1bdd0bf0',
  '3b927a1a-39ee-43dd-99bd-74e8637a703f',
  '6e08b73b-e36d-4907-aa29-276bb9ebeed2',
  'bc1a0a83-0809-4779-908e-17ee3a77fd26'],
 'embeddings': array([[-0.05562957, -0.0203272 , -0.01269856, ...,  0.00051751,
         -0.02111294, -0.01154474],
        [-0.00921848, -0.02626629, -0.03133413, ...,  0.00726502,
         -0.04161012, -0.00883735],
        [-0.03262324,  0.01831091, -0.0045399 , ...,  0.00915938,
         -0.02804055, -0.03489326],
        [-0.0146971 , -0.01844227, -0.02503126, ...,  0.01035003,
         -0.01682558, -0.01603061],
        [-0.03304848, -0.03280932, -0.02324565, ..., -0.00354889,
         -0.00507506, -0.00217535]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the most successful ca

In [ ]:
# delete document
vector_store.delete(ids=['09a39dc6-3ba6-4ea7-927e-fdda591da5e4'])

In [ ]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['2aae31a9-71fd-4f64-988b-b57b05235522',
  '879eb886-4644-4176-b93b-bbfc1bdd0bf0',
  '3b927a1a-39ee-43dd-99bd-74e8637a703f',
  '6e08b73b-e36d-4907-aa29-276bb9ebeed2',
  'bc1a0a83-0809-4779-908e-17ee3a77fd26'],
 'embeddings': array([[-0.05562957, -0.0203272 , -0.01269856, ...,  0.00051751,
         -0.02111294, -0.01154474],
        [-0.00921848, -0.02626629, -0.03133413, ...,  0.00726502,
         -0.04161012, -0.00883735],
        [-0.03262324,  0.01831091, -0.0045399 , ...,  0.00915938,
         -0.02804055, -0.03489326],
        [-0.0146971 , -0.01844227, -0.02503126, ...,  0.01035003,
         -0.01682558, -0.01603061],
        [-0.03304848, -0.03280932, -0.02324565, ..., -0.00354889,
         -0.00507506, -0.00217535]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the most successful ca